#### Import packages, set directories and parameters

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
import math
from tqdm import tqdm
from Comparative_Analysis import Utilities as util
from joblib import Parallel, delayed
from Comparative_Analysis import Blast_Functions as blastfn
import time
import os
import copy
from scipy.stats import chi2
from Comparative_Analysis import Alignment as alignfn
from Bio.Align.Applications import MuscleCommandline
import subprocess
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
project_dir = 'F:/Project_Data/mabR_Project'
mycobacteria_seq_dir = 'F:/Datasets/NCBI_Refseq_Mycobacteriaceae_Complete_Annot_20230511/data'
tb_species = 'AL123456.3' 
tb_annotation_dirname = 'GCA_000195955.2'
min_region_length = 7 
full_build = False
num_cores = 16
core_numbers = list(range(1, num_cores+1))
muscle_exe = 'C:/Users/nicho/Muscle/muscle3.8.31_i86win32.exe'
full_run = False

In [16]:
for record in (SeqIO.parse(mycobacteria_seq_dir + '/'+tb_annotation_dirname+'/genomic.gbff', "genbank")):
    accession_ver = record.annotations['accessions'][0] + '.' + str(record.annotations['sequence_version'])
    full_sequence = str(record.seq)
    for feature in record.features:
        a = feature.qualifiers
        if feature.type in ['mobile_element']:#, 'repeat_region']:
            (start, end, strand) = (int(feature.location.start), int(feature.location.end), int(feature.location.strand))
            if not(a.get("note") == None):
                product = a.get("note")[0]
            if strand == 1:
                nt_seq = full_sequence[start:end]
            else:
                nt_seq = util.reverse_complement(full_sequence[start:end])
            print(feature.type, product, start, end, strand, nt_seq)

     

mobile_element IS1536, len: 1384 nt. Partial copy of insertion sequence IS_1536. This region is a possible MT-complex-specific genomic island (See Becq et al.,2007). 701383 702767 1 GGTGAGTGGTCGGTAGTTGTCGGTGGCGTGCTGTAGGAACAGAGGTATGAATCTCGCGGCGTGGGCCGAGCGCAATGGCGTCGCGCGGGTGACCGCGTATCGCTGGTTCCACGCTGGGCTCTTGCCGGTCCCGGCCCGGAAGGTTGGTCGACTCATTCTGGTCGACGAGCTGGCTAGCGAGGCTGGCGCGCAGCCAAAGACTGCGGTGTACGCGCGGGTGTCGTCGGCTGATCAGAAGTCTGATTTGGATCGGCAGGTGGCGCGGGTGACTTCGTGGGCCACAGCCGAACAGATCCCGGTCGACAAGGTTGTCACCGAGGTCGGGTCGGTGCTCAACGGGCACCGACGTAAGTTCCCTGCGGTGCTGCGCGATCTGTCGGTCACGCGGATTGTGGTTGAGCATCGGGATCGGTTCTGCCGGTTCGGTTCGGAGTATGTCCACGCTGCGCTGGCCGCTCAGGGTCGGGAGTTGGTCGTGGTGGACTCGGCCGAGGTTGACGATGACCTGGTATGGGATATGACCGAGATTCTGACCTCGATGTGCGCAAGGTTGTATGGCAAACGTGCTGCTCAGAACCGGGCCAAGCGGGCCGTCGCGGCTGCCGCTGTCGATGATCATGAGGCGGCCTGAGATGCCGCGTTTGGAGATCCCCAACGGCTGGTGTGTGCAAGCGTTCCGGTTCACACTCGATCCGACCGCCGAGCAGGCACACGCGTTGGCGCGGCATTTCGGCGCCCGCCGCAAGGCCTACAACTGGACCGTCGCGCAGCTGAAAGCCGATATCCAAGCGTGGCGCGCGACCGGCGCCCAGACGGCG